In [1]:
import os
import random
import json
from PIL import Image

# if want to change the format --> has to change: 
#    # Set the order of classes and images within each class
#    # Set the weights for each image within each class
#    # Calculate rarity
#    # Create metadata file


# Set the directory where your images are stored
directory = 'C:/Users/ERICK ROMERO/inteligencia_artificial/Projeto-Deise-Dogs-v2/images'

# Set the order of classes and images within each class
class_order = ['background', 'body', 'tool', 'tools']

# Set the weights for each image within each class
image_weights = {
    'background': {'Beach.png': 33, 'Indoor.png': 37, 'Outer-Space.png': 30},
    'body': {'Doggy-Alien.png': 40, 'Doggy.png': 60},
    'tool': { 'Sunglasses.png': 30, 'None.png': 70},
    'tools': {'Hat.png': 30, 'None.png': 70}
}

# Set the number of final images you want to create
num_images = 10

# Set the size of the final images
small_size = (500, 500)
big_size = (1024, 1024)
final_image_sizes = [small_size, big_size]

# Set the token for the images
token = 'DEISE'

# Create a list of all the images in each class
image_lists = {}
for class_name in class_order:
    class_directory = os.path.join(directory, class_name)
    image_list = os.listdir(class_directory)
    weighted_image_list = []
    for image_name, weight in image_weights[class_name].items():
        for _ in range(weight):
            weighted_image_list.append(image_name)
    image_lists[class_name] = weighted_image_list + [image for image in image_list if image not in weighted_image_list]

# Create the final images and save them with the specified file names and resolutions
for i in range(num_images):
    # Select a random image from each class
    selected_images = {}
    for class_name in class_order:
        selected_images[class_name] = random.choice(image_lists[class_name])


# Create metadata file
for i in range(num_images):
    # Select a random image from each class
    selected_images = {}
    for class_name in class_order:
        selected_images[class_name] = random.choice(image_lists[class_name])

    # Calculate rarity
    rarity = (image_weights['background'][selected_images['background']] * image_weights['body'][selected_images['body']] * image_weights['tool'][selected_images['tool']] * image_weights['tools'][selected_images['tools']]) / 1000000
    # rarity = image_weights['background'][selected_images['background']]

    if rarity <= 2:
        level = 'platinum'
    elif 2 < rarity <= 5:
        level = 'gold'
    elif 5 < rarity < 15:
        level = 'standard'
    else:
        level = 'unknown'  

    # Combine the selected images into a final image
    for final_image_size in final_image_sizes:
        final_image = Image.new('RGB', final_image_size)
        for class_name in class_order:
            image_name = selected_images[class_name]
            image_path = os.path.join(directory, class_name, image_name)
            image = Image.open(image_path)
            if image.mode == 'RGBA':  # check if the image has a transparency channel
                mask = image.split()[3]  # get the alpha channel as a mask
                final_image.paste(image, mask=mask)
            else:
                final_image.paste(image)

        # Save the final images with the specified file names
        if final_image_size == small_size:
            file_name = f'{token} #{i+1:04}.png'
            subfile_name = None
        elif final_image_size == big_size:
            file_name = f'{token} #{i+1:04}.subfile.1.png'
            subfile_name = f'{token} #{i+1:04}.png'
        image_path = os.path.join(directory, file_name)
        final_image.save(image_path)

        # Create a subfile if necessary
        if subfile_name is not None:
            subfile_path = os.path.join(directory, subfile_name)
            final_image.resize(small_size).save(subfile_path)

        # Create metadata file
        metadata = {
            "721": {
                "<policy_id>": {
                    "<asset_name>": {
                        "name": "<display_name>",
                        "image": "<ipfs_link>",
                        "mediaType": "<mime_type>",
                        "description": "animal care + renewable energy + blockchain",
                        "social project": "linktr.ee/deisedogs",
                        "green Project": "www.intitech.com.br",
                        "blockchain Project": "www.digitaltwintech.com.br",
                        "background": selected_images['background'].replace('.png', ''),
                        "body": selected_images['body'].replace('.png', ''),
                        "tool": selected_images['tool'].replace('.png', ''),
                        "tools": selected_images['tools'].replace('.png', ''),
                        "rarity": f"{rarity:.2f}%",
                        "edition": f"{level}",
                        "files": [
                            {
                                "name": "<display_name>",
                                "mediaType": "<mime_type>",
                                "src": "<ipfs_link>"
                            }
                        ]
                    }
                },
                "version": "1.0"
            }
        }
        metadata_path = os.path.join(directory, f'{token} #{i+1:04}.metadata')
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=4)

In [2]:
import os
import random
import json
from PIL import Image

#Procedure
    #First create files, subfiles and metadata following the first code
    # On Nmaker Studio, create a project, update info for gateway payment UI. Dont modify the metadata on studio, keep it as default. Mint the token of Royalty and ID.
    #On the project created on NMAKER studio, bulk import (Drag and drop) the files and subfiles created from the code above, No need to paste the metadata. Click on the box and select png files, do 400 files at the time.
    #Export metadata from nmaker studio from the project level (Export metadata as ZIP). Paste this metadata in Visual Studio project under the folder metadata-nmaker
    #It is ok that the format of the file names are different between images and metadata-nmaker
    #MyToken #0001.metadata and MyToken0001.metadata
    #Run this second code. After running, the metadata from folder images should be updated (2 different IPFS + info). Drag and drop that metadata (image folder) into NMAKER Studio. 500 at a time.

# Define the source and destination folder paths
source_folder_path = 'C:/Users/ERICK ROMERO/inteligencia_artificial/Projeto-Deise-Dogs-v2/metadata-nmaker'
destination_folder_path = 'C:/Users/ERICK ROMERO/inteligencia_artificial/Projeto-Deise-Dogs-v2/images'

# Set the token for the images
token = 'DEISE'

# Loop through each file in the source folder
for source_filename in os.listdir(source_folder_path):
    # Construct the full source file path
    source_file_path = os.path.join(source_folder_path, source_filename)

    # Load the source file data
    with open(source_file_path, 'r') as source_file:
        source_data = json.load(source_file)

    # Extract the asset name from the source file name
    asset_name = source_filename.replace('.metadata', '').replace(token, '')

    # Extract the "files" field from the source data
    files_field = source_data['721'][list(source_data['721'].keys())[0]][f"{token}{asset_name}"]['files']

    # Construct the full destination file path
    destination_filename = source_filename.replace(token, f'{token} #')
    destination_file_path = os.path.join(destination_folder_path, destination_filename)

    # Load the destination file data
    with open(destination_file_path, 'r') as destination_file:
        destination_data = json.load(destination_file)

    # Update the "files" field in the destination data
    destination_data['721'][list(destination_data['721'].keys())[0]]['<asset_name>']['files'] = files_field

    # Save the updated destination file data
    with open(destination_file_path, 'w') as destination_file:
        json.dump(destination_data, destination_file, indent=2)
